## Sliding window analysis
This file takes the raw data from the `data_hep*.csv` files, removes any missing/NA data, prints summary statistics on the data, and then performs Analysis 1 and 2. The output of both analyses can be found in `correlations_julia.csv` and `epoch_correlations.csv` respectively.

For the plotting of graphs, see `plotting.qmd`

In [46]:
using CSV, DataFrames, HTTP, HypothesisTests, Base.Threads, Statistics, DataStructures

println(Threads.nthreads())

# Types
@kwdef struct CorTestResult
    Parameter1::Symbol = Symbol()
    Parameter2::Symbol = Symbol()
    r::Float64 = 0.0
    abs_r::Float64 = 0.0
    time::Float64 = 0.0
    condition::Symbol = Symbol()
    window_size::Float64 = 0.0
    p::Float64 = 0.0
    t::Float64 = 0.0
    ci_lower::Float64 = 0.0
    ci_higher::Float64 = 0.0
    ci::Float64 = 0.0
    n::Int = 0
    df_error::Int = 0
end

@kwdef struct EpochCorTestResult
    Parameter1::Symbol = Symbol()
    Parameter2::Symbol = Symbol()
    r::Float64 = 0.0
    abs_r::Float64 = 0.0
    p::Float64 = 0.0
    t::Float64 = 0.0
    ci_lower::Float64 = 0.0
    ci_higher::Float64 = 0.0
    ci::Float64 = 0.0
    n::Int = 0
    df_error::Int = 0
    epoch::Int = 0
end

struct EpochStats
    min_count::Int
    max_count::Int
    mean_count::Float64
    std_count::Float64
end

struct AgeStats
    min::Int
    max::Int
    mean::Float64
    std::Float64
    vec::Vector{Union{Int, Missing}}
end

struct PPStats
    included::Vector{String7}
    included_count::Int
    excluded::Vector{String7}
    excluded_count::Int
    total::Int
    age::AgeStats
    gender::Dict

    function PPStats(included::Vector{String7}, excluded::Vector{String7}, age::AgeStats, gender::Dict)
        unique_included = unique(included)
        unique_excluded = unique(excluded)
        total = length(unique_included) + length(unique_excluded)
        new(unique_included, length(unique_included), unique_excluded, length(unique_excluded), total, age, gender)
    end
end

# Utility Functions

filter_missings(args...) = collect.(skipmissings(args...)) # Evaluate the iterator from `skipmissings` ahead of time in order to satisfy type checking.

remove_undef(vec) = [vec[i] for i = 1:length(vec) if isassigned(vec, i)] # Remove any undef

6


remove_undef (generic function with 1 method)

In [47]:
# Calculate summary statistics
function summary_stats(filtered_intero, nomissing, pp_population_stats, hepdata)
    # Create a dataframe showing participants with any columns containing missing data. These pps are excluded.
    global missingdf = filter(row -> any(ismissing, row) && row.Participant in hepdata.Participant, filtered_intero)
    cols_to_remove = [col for col in names(missingdf) if all(!ismissing, missingdf[!, col])] # Remove columns with no missing data, for visualisation purposes
    select!(missingdf, :Participant, Not(cols_to_remove))

    included_participants = unique(nomissing.Participant)
    excluded_participants = unique(missingdf.Participant)

    filter!(row -> row.Participant in included_participants, pp_population_stats)
    age_vec = skipmissing(pp_population_stats.Age)

    agestats = AgeStats(minimum(age_vec), maximum(age_vec), mean(age_vec), std(age_vec), pp_population_stats.Age)
    genderstats = Dict(counter(pp_population_stats.Gender))
    global ppstats = PPStats(included_participants, excluded_participants, agestats, genderstats)

    println("Beginning analysis with $(nrow(nomissing)) time points.")
    println("Excluded $(ppstats.excluded_count) participants: $(join(ppstats.excluded, ", "))")
end


summary_stats (generic function with 1 method)

In [48]:
function load_csvs(first::Union{Int, Nothing} = nothing)
    csvs = filter(file -> contains(file, "data_hep"), readdir("../data", join = true))
    hepdata = reduce(
        vcat,
        [CSV.read(filepath, DataFrame, types = Dict(:Condition => Symbol), missingstring = "NA") for filepath in csvs],
    )

    ppreq = HTTP.get(
        "https://raw.githubusercontent.com/RealityBending/InteroceptionPrimals/main/data/data_participants.csv",
    )
    interoprimals = DataFrame(CSV.File(ppreq.body, missingstring = "NA"), copycols = false)
    rename!(interoprimals, :participant_id => :Participant)

    # Copy a slice for pp stats for later
    pp_population_stats = interoprimals[:, [:Participant, :Age, :Gender]]

    # We only care about MAIA, IAS, HRV and HCT, remove the rest.
    filtered_cols = filter(
        col ->
            !contains(col, r"\d") && (
                col == "Participant" ||
                startswith(col, "MAIA_") ||
                startswith(col, "IAS") ||
                startswith(col, "HRV") ||
                startswith(col, "HCT")
            ),
        names(interoprimals),
    )

    filtered_intero = (@view interoprimals[!, filtered_cols])

    joined = innerjoin(hepdata, filtered_intero, on = :Participant)
    nomissing = dropmissing(joined)

    summary_stats(filtered_intero, nomissing, pp_population_stats, hepdata)

    if (first isa Int)
        return first(nomissing, first)
    else
        return nomissing
    end
end

df = load_csvs()

println("PPs included: $(ppstats.included_count)")
println("PPs excluded: $(ppstats.excluded_count)")
println("Min age: $(ppstats.age.min)")
println("Max age: $(ppstats.age.max)")
println("Mean age: $(ppstats.age.mean)")
println("SD age: $(ppstats.age.std)")
println("Gender count: $(ppstats.gender)")

df


Beginning analysis with 9255411 time points.
Excluded 13 participants: sub-07, sub-09, sub-100, sub-11, sub-110, sub-114, sub-33, sub-38, sub-41, sub-48, sub-67, sub-72, sub-90
PPs included: 105
PPs excluded: 13
Min age: 18
Max age: 50
Mean age: 21.25
SD age: 4.878623870907286
Gender count: Dict{String7, Int64}("Female" => 84, "Male" => 16, "Other" => 3, "" => 2)


Row,time,epoch,AF7,AF8,PPG_Muse,ECG,RSP,Participant,Condition,MAIA_Noticing,MAIA_NotDistracting,MAIA_NotWorrying,MAIA_AttentionRegulation,MAIA_EmotionalAwareness,MAIA_SelfRegulation,MAIA_BodyListening,MAIA_Trusting,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SampEn,HRV_HF,HRV_HFD,HRV_LFHF,HRV_IALS,HRV_Ca,HRV_AI,HCT_Accuracy,HCT_Sensibility,HCT_Awareness,HRV_MeanNN_HCT,HRV_SDNN_HCT,HRV_RMSSD_HCT,IAS_Total
,Float64,Int64,Float64,Float64,Float64,Float64,Float64,String7,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.4,14,-1.66513,-51.3104,1.0878e5,-0.029,-0.132573,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
2,-0.39,14,-45.9361,-97.5502,1.08789e5,-0.029,-0.135,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
3,-0.38,14,-50.9865,-94.3602,1.08788e5,-0.0314266,-0.135,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
4,-0.37,14,-53.9315,-109.773,1.08796e5,-0.018,-0.139,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
5,-0.36,14,-65.2803,-106.944,1.08808e5,-0.0213826,-0.142,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
6,-0.35,14,-33.6722,-63.2689,1.08828e5,-0.021,-0.142,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
7,-0.34,14,-49.3368,-78.5769,1.08848e5,-0.021,-0.142,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
8,-0.33,14,-62.7101,-89.4344,1.08862e5,-0.023,-0.142,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476
9,-0.32,14,-30.315,-60.7955,108850.0,-0.018,-0.142,sub-91,RestingState,0.705,0.74,0.788,0.297143,0.872,0.385,0.673333,0.793333,724.113,73.5942,66.5277,1.38837,0.0274694,1.8849,0.366189,0.376147,0.483262,50.4572,0.768808,0.0216667,0.294245,809.041,82.2469,76.5381,0.840476


In [49]:
missingdf # Show participants with missing data, and the corresponding columns with missing data

Row,Participant,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SampEn,HRV_HF,HRV_HFD,HRV_LFHF,HRV_IALS,HRV_Ca,HRV_AI,HCT_Accuracy,HCT_Sensibility,HCT_Awareness,HRV_MeanNN_HCT,HRV_SDNN_HCT,HRV_RMSSD_HCT,IAS_Total
,String7,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,sub-07,708.985,52.0111,23.0087,1.04182,0.0022474,1.49104,5.67338,0.345808,0.563756,50.2206,0.60546,0.81,missing,704.238,46.5677,20.8206,0.919048
2,sub-09,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,0.75341,0.216667,0.115954,missing,missing,missing,0.587619
3,sub-100,491.417,39.5178,13.0882,0.392934,0.00434032,1.26727,7.54702,0.200426,0.587454,50.0429,0.0,0.0816667,missing,502.174,27.8504,9.60765,0.739524
4,sub-11,624.518,27.3093,14.613,1.39289,0.00434389,1.57621,4.37752,0.520694,0.543171,50.1477,0.834906,0.666667,missing,707.287,34.385,24.0801,0.824762
5,sub-110,958.072,73.2936,75.6814,1.67819,0.0301215,2.01237,0.192793,0.435223,0.533411,50.2074,0.0,0.0116667,missing,1000.7,99.423,95.3351,0.60381
6,sub-114,882.94,61.9989,50.1993,1.68785,0.00495871,1.86556,1.84788,0.662289,0.523838,50.0545,0.0206948,0.751667,missing,893.644,98.1567,73.657,0.598571
7,sub-33,607.093,43.0798,17.8338,1.01346,0.0012084,1.53901,4.37716,0.45232,0.522945,49.8718,0.565444,0.0,missing,649.494,55.7509,27.7514,missing
8,sub-38,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,0.855093,0.395,-0.0857143,missing,missing,missing,0.578095
9,sub-41,670.72,74.418,36.9261,0.751537,0.00174751,1.52362,11.9403,0.216102,0.59101,50.0689,0.627243,0.0,missing,729.045,49.0043,27.0533,0.670476


In [50]:
function prepare_variables(windowed)

    # What do I groupby? Only participant has good r but bad p, Participant and epoch has bad r but good p!
    time_grouped = groupby(windowed, [:time, :Participant])

    time_combined = combine(
        time_grouped,
        [:AF7, :AF8] .=> mean .=> [:AF7_Mean, :AF8_Mean],
        Cols(startswith("MAIA")) .=> first,
        Cols(startswith("IAS")) .=> first,
        Cols(startswith("HRV")) .=> first,
        Cols(startswith("HCT")) .=> first,
        renamecols = false,
    )

    participant_grouped = groupby(time_combined, :Participant)

    participant_combined = combine(
        participant_grouped,
        [:AF7_Mean, :AF8_Mean] .=> mean,
        Cols(startswith("MAIA")) .=> first,
        Cols(startswith("IAS")) .=> first,
        Cols(startswith("HRV")) .=> first,
        Cols(startswith("HCT")) .=> first,
        renamecols = false,
    )

    means =
        eachcol(select(participant_combined, [:AF7_Mean, :AF8_Mean], copycols = false))

    interoceptive = eachcol(
        select(
            participant_combined,
            Cols(startswith("MAIA"), startswith("IAS"), startswith("HRV"), startswith("HCT")),
            copycols = false,
        ),
    )

    return (means, interoceptive)
end


prepare_variables (generic function with 1 method)

In [51]:
function pairwise_correlation(
    means,
    interoceptive,
    output,
    prealloc_i;
    i::Union{Float64, Nothing} = nothing,
    c::Union{Symbol, Nothing} = nothing,
    w::Union{Float64, Nothing} = nothing,
    e::Union{Int, Nothing} = nothing,
)

    for (Parameter1, cols1) in pairs(means)
        for (Parameter2, cols2) in pairs(interoceptive) # Correlate the AF7/8 means against each interoceptive measure

            (cols1_filtered, cols2_filtered) = filter_missings(cols1, cols2)

            if ((length(cols1_filtered)) === 0 || (length(cols1_filtered)) === 0)
                continue
            end

            cor = HypothesisTests.CorrelationTest(cols1_filtered, cols2_filtered)

            if (cor.r ≈ 1.0 || cor.r ≈ -1.0 || isnan(cor.r) || cor.t ≈ Inf || cor.t ≈ -Inf)
                # Assume something's gone wrong
                continue
            end

            pvalue = HypothesisTests.pvalue(cor)
            (ci_lower, ci_higher) = HypothesisTests.confint(cor)

            if (e === nothing) # If sliding window

                output[prealloc_i[]] = CorTestResult(
                    Parameter1 = Parameter1,
                    Parameter2 = Parameter2,
                    r = cor.r,
                    abs_r = abs(cor.r),
                    time = i,
                    condition = c,
                    window_size = w,
                    p = pvalue,
                    t = cor.t,
                    ci_lower = ci_lower,
                    ci_higher = ci_higher,
                    ci = 0.95,
                    n = cor.n,
                    df_error = cor.n - 2, # n minus number of variables
                )

            else # If epoch analysis
                output[prealloc_i[]] = EpochCorTestResult(
                    Parameter1 = Parameter1,
                    Parameter2 = Parameter2,
                    r = cor.r,
                    abs_r = abs(cor.r),
                    p = pvalue,
                    t = cor.t,
                    ci_lower = ci_lower,
                    ci_higher = ci_higher,
                    ci = 0.95,
                    n = cor.n,
                    df_error = cor.n - 2, # n minus number of variables
                    epoch = e,
                )
            end

            atomic_add!(prealloc_i, 1)

        end
    end
end

pairwise_correlation (generic function with 1 method)

In [52]:
# 0.05 means 0.05 each side of the time point (total width 0.1)
# Times are fractions of seconds, so 0.05 = 50ms
window_widths = range(start = 0.05, stop = 0.2, step = 0.025)
times = range(start = -0.4, stop = 0.8, step = 0.01)
conditions = [:HCT, :RestingState]

steps = length(window_widths) * length(conditions) * length(times) * 102 # 100 correlations for each combination, plus 2 for good luck

output = Vector{CorTestResult}(undef, steps)
prealloc_i = Threads.Atomic{Int}(1)

Threads.@threads for w in window_widths
    for c in conditions
        for i in times

            if (i - w < -0.4 || i + w > 0.8)
                continue
            end

            windowed = (@view df[(df.time .>= (i - w) .&& df.time .< (i + w)) .&& df.Condition .=== c, :])

            if (nrow(windowed) === 0)
                # println("Thread $(Threads.threadid()): SKIPPED $i, $c, $w")
                continue
            end

            (means, interoceptive) = prepare_variables(windowed)

            pairwise_correlation(means, interoceptive, output, prealloc_i; i = i, c = c, w = w)

            # println("Thread $(Threads.threadid()): $i, $w, $c")
        end
    end
end

global window_df = DataFrame(remove_undef(output))

Row,Parameter1,Parameter2,r,abs_r,time,condition,window_size,p,t,ci_lower,ci_higher,ci,n,df_error
,Symbol,Symbol,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,AF7_Mean,MAIA_Noticing,-0.0467988,0.0467988,-0.22,HCT,0.175,0.637108,-0.473163,-0.237249,0.147115,0.95,104,102
2,AF7_Mean,MAIA_Noticing,-0.211245,0.211245,-0.3,HCT,0.1,0.03135,-2.18273,-0.388046,-0.0194479,0.95,104,102
3,AF7_Mean,MAIA_Noticing,0.0520345,0.0520345,-0.2,HCT,0.2,0.599867,0.526235,-0.141977,0.242196,0.95,104,102
4,AF7_Mean,MAIA_NotDistracting,-0.000686891,0.000686891,-0.22,HCT,0.175,0.994478,-0.00693726,-0.19325,0.191927,0.95,104,102
5,AF7_Mean,MAIA_NotDistracting,-0.12153,0.12153,-0.3,HCT,0.1,0.219092,-1.23656,-0.306934,0.0727615,0.95,104,102
6,AF7_Mean,MAIA_NotWorrying,0.0532911,0.0532911,-0.22,HCT,0.175,0.591076,0.538979,-0.140742,0.243381,0.95,104,102
7,AF7_Mean,MAIA_NotWorrying,0.119953,0.119953,-0.3,HCT,0.1,0.225174,1.22028,-0.074353,0.305484,0.95,104,102
8,AF7_Mean,MAIA_AttentionRegulation,-0.0690549,0.0690549,-0.22,HCT,0.175,0.486087,-0.699089,-0.258209,0.125198,0.95,104,102
9,AF7_Mean,MAIA_EmotionalAwareness,0.0103325,0.0103325,-0.22,HCT,0.175,0.91709,0.104359,-0.182619,0.202518,0.95,104,102


In [53]:
# Count how many significant correlations there are per mean/index

significant = (@view window_df[window_df.p .< 0.05, :])

significance_count_total = combine(groupby(significant, [:Parameter2]), nrow => :count) # How many significant correlations in total

window_significance_count = combine(groupby(significant, [:Parameter2, :window_size]), nrow => :count) # How many per window size

condition_count = combine(groupby(significant, [:Parameter2, :condition]), nrow => :count) # How many per condition

window_condition_count = combine(groupby(significant, [:Parameter2, :window_size, :condition]), nrow => :count) # How many per condition

sort!(significance_count_total, :count, rev=true)

Row,Parameter2,count
,Symbol,Int64
1,MAIA_EmotionalAwareness,1007
2,HRV_LFHF,749
3,MAIA_Noticing,629
4,MAIA_BodyListening,470
5,MAIA_AttentionRegulation,199
6,IAS_Total,172
7,MAIA_SelfRegulation,169
8,HRV_HF,166
9,MAIA_NotDistracting,143


In [28]:
sort!(window_significance_count, [:count, :window_size], rev=true)

Row,Parameter2,window_size,count
,Symbol,Float64,Int64
1,MAIA_EmotionalAwareness,0.05,322
2,MAIA_EmotionalAwareness,0.075,321
3,MAIA_EmotionalAwareness,0.1,307
4,MAIA_EmotionalAwareness,0.125,297
5,MAIA_EmotionalAwareness,0.15,278
6,MAIA_EmotionalAwareness,0.175,232
7,MAIA_EmotionalAwareness,0.2,218
8,MAIA_Noticing,0.05,206
9,MAIA_BodyListening,0.05,199


In [ ]:
sort!(condition_count, :count, rev=true)

"window_condition_count.csv"

In [ ]:
sort!(window_condition_count, [:count, :window_size], rev=true)

Row,Parameter2,window_size,condition,count
,Symbol,Float64,Symbol,Int64
1,HRV_LFHF,0.2,HCT,128
2,HRV_LFHF,0.175,HCT,128
3,HRV_LFHF,0.15,HCT,119
4,HRV_LFHF,0.1,HCT,106
5,HRV_LFHF,0.125,HCT,101
6,HRV_LFHF,0.075,HCT,93
7,MAIA_EmotionalAwareness,0.075,RestingState,85
8,MAIA_Noticing,0.075,HCT,83
9,MAIA_Noticing,0.1,HCT,82


In [54]:
function epoch_stats(df)

    grouped = groupby(df, :Participant)
    epoch_summary = combine(grouped, :epoch => maximum => :epoch_count)

    min_count = minimum(epoch_summary.epoch_count)
    max_count = maximum(epoch_summary.epoch_count)
    mean_count = mean(epoch_summary.epoch_count)
    std_count = std(epoch_summary.epoch_count)

    return EpochStats(min_count, max_count, mean_count, std_count)
end

epoch_stats (generic function with 1 method)

In [58]:
global epstats = epoch_stats(df)

epochs = 1:(epstats.max_count)

steps = length(epochs) * 102 # 50 correlations for each combination, plus 2 for good luck

output = Vector{EpochCorTestResult}(undef, steps)
prealloc_i = Threads.Atomic{Int}(1)

Threads.@threads for e in epochs

    windowed = (@view df[df.epoch .<= e, :])

    if (nrow(windowed) === 0)
        continue
    end

    (means, interoceptive) = prepare_variables(windowed)

    pairwise_correlation(means, interoceptive, output, prealloc_i; e = e)

    # println("Thread $(Threads.threadid()): $e")


end
global epoch_df = DataFrame(remove_undef(output))

Row,Parameter1,Parameter2,r,abs_r,p,t,ci_lower,ci_higher,ci,n,df_error,epoch
,Symbol,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,AF7_Mean,MAIA_Noticing,0.203346,0.203346,0.0469126,2.01358,0.00298151,0.388013,0.95,96,94,1
2,AF7_Mean,MAIA_NotDistracting,0.179335,0.179335,0.0804118,1.76737,-0.0219399,0.366639,0.95,96,94,1
3,AF7_Mean,MAIA_NotWorrying,-0.0252692,0.0252692,0.806935,-0.245072,-0.224617,0.176109,0.95,96,94,1
4,AF7_Mean,MAIA_AttentionRegulation,0.200858,0.200858,0.0497313,1.98791,0.000387926,0.385808,0.95,96,94,1
5,AF7_Mean,MAIA_EmotionalAwareness,0.335955,0.335955,0.000818764,3.4582,0.145252,0.502589,0.95,96,94,1
6,AF7_Mean,MAIA_SelfRegulation,0.138443,0.138443,0.178569,1.3553,-0.0638145,0.329775,0.95,96,94,1
7,AF7_Mean,MAIA_BodyListening,0.324663,0.324663,0.00124984,3.328,0.132822,0.493055,0.95,96,94,1
8,AF7_Mean,MAIA_Trusting,0.0616587,0.0616587,0.550652,0.598943,-0.140565,0.258944,0.95,96,94,1
9,AF7_Mean,IAS_Total,0.0344597,0.0344597,0.738899,0.334298,-0.167181,0.233334,0.95,96,94,1
